#### 1. Dataset Loading and Exploration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from scipy.io import loadmat
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import joblib

# Load MPII dataset annotations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from scipy.io import loadmat
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import joblib

# Load MPII dataset annotations
def load_mpii_dataset(annot_file, img_dir):
    print(f"Loading annotations from {annot_file}")
    annotations = loadmat(annot_file)
    
    # Extract relevant data
    annotations = annotations['RELEASE']
    img_names = annotations['annolist'][0, 0][0]
    joint_annotations = annotations['annolist'][0, 0][1]
    is_train = annotations['img_train'][0, 0][0]
    
    data = []
    valid_count = 0
    
    for i in range(len(img_names)):
        if is_train[i]:  # Only use training data
            try:
                img_name = str(img_names[i][0][0][0])
                joints = joint_annotations[i][0][0]
                
                if len(joints) > 0 and 'x' in dir(joints[0][0]):
                    joint_coords = []
                    for j in range(joints.shape[0]):
                        if joints[j][0].size > 0:
                            x = float(joints[j][0]['x'][0, 0])
                            y = float(joints[j][0]['y'][0, 0])
                            joint_coords.append((x, y))
                        else:
                            joint_coords.append(None)
                    
                    img_path = os.path.join(img_dir, img_name)
                    if os.path.exists(img_path):
                        data.append({
                            'img_path': img_path,
                            'joints': joint_coords
                        })
                        valid_count += 1
                        # Limit to a reasonable number for initial testing
                        if valid_count >= 500:
                            break
            except Exception as e:
                continue
    
    print(f"Loaded {len(data)} valid annotations")
    return data

# Preprocess data for HOG+SVM
def preprocess_for_hog_svm(data):
    processed_data = []
    
    for item in data:
        img = cv2.imread(item['img_path'])
        if img is None:
            continue
            
        # Get bounding box from joints
        valid_joints = [j for j in item['joints'] if j is not None]
        if not valid_joints:
            continue
            
        x_coords = [j[0] for j in valid_joints]
        y_coords = [j[1] for j in valid_joints]
        
        # Get bounding box with padding
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        
        # Add padding
        width = max(1, x_max - x_min)  # Avoid division by zero
        height = max(1, y_max - y_min)
        padding_x = width * 0.2
        padding_y = height * 0.2
        
        x_min = max(0, x_min - padding_x)
        x_max = min(img.shape[1], x_max + padding_x)
        y_min = max(0, y_min - padding_y)
        y_max = min(img.shape[0], y_max + padding_y)
        
        # Crop person
        person_img = img[int(y_min):int(y_max), int(x_min):int(x_max)]
        
        # Resize for consistent HOG feature extraction
        try:
            resized_img = cv2.resize(person_img, (128, 256))
            
            # Store original joint coordinates relative to the cropped image
            adjusted_joints = []
            for joint in item['joints']:
                if joint is not None:
                    # Scale joint coordinates to the resized image
                    adj_x = (joint[0] - x_min) * 128 / (x_max - x_min)
                    adj_y = (joint[1] - y_min) * 256 / (y_max - y_min)
                    adjusted_joints.append((adj_x, adj_y))
                else:
                    adjusted_joints.append(None)
            
            processed_data.append({
                'image': resized_img,
                'joints': adjusted_joints,
                'original_path': item['img_path']
            })
        except Exception as e:
            print(f"Error processing image {item['img_path']}: {e}")
    
    print(f"Processed {len(processed_data)} images")
    return processed_data

# Extract HOG features
def extract_hog_features(processed_data):
    # HOG parameters
    win_size = (128, 256)
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    nbins = 9
    
    # Initialize HOG descriptor
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    
    features = []
    labels = []
    
    for i, item in enumerate(processed_data):
        print(f"Extracting HOG features for image {i+1}/{len(processed_data)}...", end="\r")
        
        img_gray = cv2.cvtColor(item['image'], cv2.COLOR_BGR2GRAY)
        
        # Extract HOG features
        hog_features = hog.compute(img_gray)
        features.append(hog_features.flatten())
        
        # Format joints as labels
        flat_joints = []
        for joint in item['joints']:
            if joint is not None:
                flat_joints.extend([joint[0], joint[1]])
            else:
                flat_joints.extend([0, 0])  # Use 0,0 for missing joints
        
        labels.append(np.array(flat_joints))
    
    print("\nFeature extraction complete!")
    return np.array(features), np.array(labels)

# Train SVR models for each joint
def train_joint_models(X_train, y_train, X_test, y_test, num_joints=16):
    joint_models = []
    joint_errors = []
    
    # Calculate the number of joints from the data
    num_joints = y_train.shape[1] // 2
    
    start_time = time.time()
    
    # Train a separate model for each joint (x,y) coordinate
    for i in range(num_joints):
        print(f"Training model for joint {i+1}/{num_joints}...")
        
        # X-coordinate model
        x_model = SVR(kernel='rbf', C=10, gamma='scale')
        x_model.fit(X_train, y_train[:, i*2])
        
        # Y-coordinate model
        y_model = SVR(kernel='rbf', C=10, gamma='scale')
        y_model.fit(X_train, y_train[:, i*2+1])
        
        # Validate
        x_pred = x_model.predict(X_test)
        y_pred = y_model.predict(X_test)
        
        # Calculate error
        x_mse = mean_squared_error(y_test[:, i*2], x_pred)
        y_mse = mean_squared_error(y_test[:, i*2+1], y_pred)
        
        joint_models.append((x_model, y_model))
        joint_errors.append((x_mse, y_mse))
        
        print(f"  Joint {i+1} X MSE: {x_mse:.4f}, Y MSE: {y_mse:.4f}")
    
    elapsed_time = time.time() - start_time
    print(f"Training completed in {elapsed_time:.2f} seconds")
    
    return joint_models, joint_errors

# Evaluate and visualize results
def evaluate_and_visualize(processed_data, joint_models, feature_scaler, num_samples=5):
    # Choose random samples for visualization
    sample_indices = np.random.choice(len(processed_data), min(num_samples, len(processed_data)), replace=False)
    
    for idx in sample_indices:
        img = processed_data[idx]['image']
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Extract HOG features
        hog = cv2.HOGDescriptor((128, 256), (16, 16), (8, 8), (8, 8), 9)
        features = hog.compute(img_gray).flatten()
        
        # Scale features
        features_scaled = feature_scaler.transform(features.reshape(1, -1))
        
        # Predict joint positions
        predicted_joints = []
        for x_model, y_model in joint_models:
            x_pred = x_model.predict(features_scaled)[0]
            y_pred = y_model.predict(features_scaled)[0]
            predicted_joints.append((x_pred, y_pred))
        
        # Visualize
        plt.figure(figsize=(12, 6))
        
        # Original image with ground truth
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        for joint in processed_data[idx]['joints']:
            if joint is not None:
                plt.plot(joint[0], joint[1], 'go', markersize=5)
        plt.title('Ground Truth')
        
        # Image with predictions
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        for joint in predicted_joints:
            plt.plot(joint[0], joint[1], 'ro', markersize=5)
        plt.title('Predictions')
        
        plt.tight_layout()
        plt.show()
        
        # Draw skeleton for better visualization
        plt.figure(figsize=(8, 16))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        # Draw predicted joints
        for joint in predicted_joints:
            plt.plot(joint[0], joint[1], 'ro', markersize=5)
        
        # Define skeleton connections - simplified MPII skeleton
        # Adjust based on your joint ordering in MPII dataset
        skeleton_pairs = [
            (0, 1),  # head to neck
            (1, 2),  # neck to right shoulder
            (1, 5),  # neck to left shoulder
            (2, 3),  # right shoulder to right elbow
            (3, 4),  # right elbow to right wrist
            (5, 6),  # left shoulder to left elbow
            (6, 7),  # left elbow to left wrist
            (1, 8),  # neck to hip
            (8, 9),  # hip to right hip
            (8, 12), # hip to left hip
            (9, 10), # right hip to right knee
            (10, 11), # right knee to right ankle
            (12, 13), # left hip to left knee
            (13, 14)  # left knee to left ankle
        ]
        
        # Draw skeleton lines connecting joints
        for pair in skeleton_pairs:
            if pair[0] < len(predicted_joints) and pair[1] < len(predicted_joints):
                plt.plot([predicted_joints[pair[0]][0], predicted_joints[pair[1]][0]],
                         [predicted_joints[pair[0]][1], predicted_joints[pair[1]][1]], 'b-', linewidth=2)
        
        plt.title('Pose Skeleton')
        plt.tight_layout()
        plt.show()

# Function to predict pose on new images
def predict_pose(image_path, joint_models, feature_scaler):
    # Load and preprocess image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image from {image_path}")
        return None
    
    # Resize for consistent HOG extraction
    img_resized = cv2.resize(img, (128, 256))
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    
    # Extract HOG features
    hog = cv2.HOGDescriptor((128, 256), (16, 16), (8, 8), (8, 8), 9)
    features = hog.compute(img_gray).flatten()
    
    # Scale features
    features_scaled = feature_scaler.transform(features.reshape(1, -1))
    
    # Predict joint positions
    predicted_joints = []
    for x_model, y_model in joint_models:
        x_pred = x_model.predict(features_scaled)[0]
        y_pred = y_model.predict(features_scaled)[0]
        predicted_joints.append((x_pred, y_pred))
    
    # Visualize
    plt.figure(figsize=(8, 16))
    plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
    
    # Draw predicted joints
    for joint in predicted_joints:
        plt.plot(joint[0], joint[1], 'ro', markersize=5)
    
    # Draw skeleton lines connecting joints
    skeleton_pairs = [
        (0, 1),  # head to neck
        (1, 2),  # neck to right shoulder
        (1, 5),  # neck to left shoulder
        (2, 3),  # right shoulder to right elbow
        (3, 4),  # right elbow to right wrist
        (5, 6),  # left shoulder to left elbow
        (6, 7),  # left elbow to left wrist
        (1, 8),  # neck to hip
        (8, 9),  # hip to right hip
        (8, 12), # hip to left hip
        (9, 10), # right hip to right knee
        (10, 11), # right knee to right ankle
        (12, 13), # left hip to left knee
        (13, 14)  # left knee to left ankle
    ]
    
    for pair in skeleton_pairs:
        if pair[0] < len(predicted_joints) and pair[1] < len(predicted_joints):
            plt.plot([predicted_joints[pair[0]][0], predicted_joints[pair[1]][0]],
                     [predicted_joints[pair[0]][1], predicted_joints[pair[1]][1]], 'b-', linewidth=2)
    
    plt.title('Pose Estimation')
    plt.tight_layout()
    plt.show()
    
    return predicted_joints

# Function to save models
def save_models(joint_models, feature_scaler, filepath='hog_svm_pose_model.pkl'):
    with open(filepath, 'wb') as f:
        pickle.dump({
            'joint_models': joint_models,
            'feature_scaler': feature_scaler
        }, f)
    print(f"Models saved to {filepath}")

# Function to load models
def load_models(filepath='hog_svm_pose_model.pkl'):
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data['joint_models'], data['feature_scaler']

# Main execution
if __name__ == "__main__":
    import time
    import pickle
    
    # 1. Load MPII dataset
    print("Loading MPII dataset...")
    mpii_data = load_mpii_dataset('mpii_human_pose_v1_u12_1.mat', './') 
    
    # Check if we got any data
    if len(mpii_data) == 0:
        print("No data loaded from MPII dataset. Check paths and file format.")
    else:
        # 2. Preprocess data
        print("\nPreprocessing images...")
        processed_mpii = preprocess_for_hog_svm(mpii_data)
        
        # 3. Extract HOG features
        print("\nExtracting HOG features...")
        X, y = extract_hog_features(processed_mpii)
        print(f"Feature matrix shape: {X.shape}")
        print(f"Labels matrix shape: {y.shape}")
        
        # 4. Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # 5. Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # 6. Train models
        print("\nTraining SVM models...")
        joint_models, errors = train_joint_models(X_train_scaled, y_train, X_test_scaled, y_test)
        
        # 7. Save models
        save_models(joint_models, scaler)
        
        # 8. Evaluate
        print("\nEvaluating models...")
        evaluate_and_visualize(processed_mpii, joint_models, scaler, num_samples=3)
        
        # 9. Test on specific image
        print("\nTesting on a new image...")
        test_img_path = processed_mpii[0]['original_path']  # Using first image as example
        predict_pose(test_img_path, joint_models, scaler)

# Preprocess data for HOG+SVM
def preprocess_for_hog_svm(data):
    processed_data = []
    
    for item in data:
        img = cv2.imread(item['img_path'])
        if img is None:
            continue
            
        # Get bounding box from joints
        valid_joints = [j for j in item['joints'] if j is not None]
        if not valid_joints:
            continue
            
        x_coords = [j[0] for j in valid_joints]
        y_coords = [j[1] for j in valid_joints]
        
        # Get bounding box with padding
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        
        # Add padding
        width = max(1, x_max - x_min)  # Avoid division by zero
        height = max(1, y_max - y_min)
        padding_x = width * 0.2
        padding_y = height * 0.2
        
        x_min = max(0, x_min - padding_x)
        x_max = min(img.shape[1], x_max + padding_x)
        y_min = max(0, y_min - padding_y)
        y_max = min(img.shape[0], y_max + padding_y)
        
        # Crop person
        person_img = img[int(y_min):int(y_max), int(x_min):int(x_max)]
        
        # Resize for consistent HOG feature extraction
        try:
            resized_img = cv2.resize(person_img, (128, 256))
            
            # Store original joint coordinates relative to the cropped image
            adjusted_joints = []
            for joint in item['joints']:
                if joint is not None:
                    # Scale joint coordinates to the resized image
                    adj_x = (joint[0] - x_min) * 128 / (x_max - x_min)
                    adj_y = (joint[1] - y_min) * 256 / (y_max - y_min)
                    adjusted_joints.append((adj_x, adj_y))
                else:
                    adjusted_joints.append(None)
            
            processed_data.append({
                'image': resized_img,
                'joints': adjusted_joints,
                'original_path': item['img_path']
            })
        except Exception as e:
            print(f"Error processing image {item['img_path']}: {e}")
    
    print(f"Processed {len(processed_data)} images")
    return processed_data

# Extract HOG features
def extract_hog_features(processed_data):
    # HOG parameters
    win_size = (128, 256)
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    nbins = 9
    
    # Initialize HOG descriptor
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    
    features = []
    labels = []
    
    for i, item in enumerate(processed_data):
        print(f"Extracting HOG features for image {i+1}/{len(processed_data)}...", end="\r")
        
        img_gray = cv2.cvtColor(item['image'], cv2.COLOR_BGR2GRAY)
        
        # Extract HOG features
        hog_features = hog.compute(img_gray)
        features.append(hog_features.flatten())
        
        # Format joints as labels
        flat_joints = []
        for joint in item['joints']:
            if joint is not None:
                flat_joints.extend([joint[0], joint[1]])
            else:
                flat_joints.extend([0, 0])  # Use 0,0 for missing joints
        
        labels.append(np.array(flat_joints))
    
    print("\nFeature extraction complete!")
    return np.array(features), np.array(labels)

# Train SVR models for each joint
def train_joint_models(X_train, y_train, X_test, y_test, num_joints=16):
    joint_models = []
    joint_errors = []
    
    # Calculate the number of joints from the data
    num_joints = y_train.shape[1] // 2
    
    start_time = time.time()
    
    # Train a separate model for each joint (x,y) coordinate
    for i in range(num_joints):
        print(f"Training model for joint {i+1}/{num_joints}...")
        
        # X-coordinate model
        x_model = SVR(kernel='rbf', C=10, gamma='scale')
        x_model.fit(X_train, y_train[:, i*2])
        
        # Y-coordinate model
        y_model = SVR(kernel='rbf', C=10, gamma='scale')
        y_model.fit(X_train, y_train[:, i*2+1])
        
        # Validate
        x_pred = x_model.predict(X_test)
        y_pred = y_model.predict(X_test)
        
        # Calculate error
        x_mse = mean_squared_error(y_test[:, i*2], x_pred)
        y_mse = mean_squared_error(y_test[:, i*2+1], y_pred)
        
        joint_models.append((x_model, y_model))
        joint_errors.append((x_mse, y_mse))
        
        print(f"  Joint {i+1} X MSE: {x_mse:.4f}, Y MSE: {y_mse:.4f}")
    
    elapsed_time = time.time() - start_time
    print(f"Training completed in {elapsed_time:.2f} seconds")
    
    return joint_models, joint_errors

# Evaluate and visualize results
def evaluate_and_visualize(processed_data, joint_models, feature_scaler, num_samples=5):
    # Choose random samples for visualization
    sample_indices = np.random.choice(len(processed_data), min(num_samples, len(processed_data)), replace=False)
    
    for idx in sample_indices:
        img = processed_data[idx]['image']
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Extract HOG features
        hog = cv2.HOGDescriptor((128, 256), (16, 16), (8, 8), (8, 8), 9)
        features = hog.compute(img_gray).flatten()
        
        # Scale features
        features_scaled = feature_scaler.transform(features.reshape(1, -1))
        
        # Predict joint positions
        predicted_joints = []
        for x_model, y_model in joint_models:
            x_pred = x_model.predict(features_scaled)[0]
            y_pred = y_model.predict(features_scaled)[0]
            predicted_joints.append((x_pred, y_pred))
        
        # Visualize
        plt.figure(figsize=(12, 6))
        
        # Original image with ground truth
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        for joint in processed_data[idx]['joints']:
            if joint is not None:
                plt.plot(joint[0], joint[1], 'go', markersize=5)
        plt.title('Ground Truth')
        
        # Image with predictions
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        for joint in predicted_joints:
            plt.plot(joint[0], joint[1], 'ro', markersize=5)
        plt.title('Predictions')
        
        plt.tight_layout()
        plt.show()
        
        # Draw skeleton for better visualization
        plt.figure(figsize=(8, 16))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        
        # Draw predicted joints
        for joint in predicted_joints:
            plt.plot(joint[0], joint[1], 'ro', markersize=5)
        
        # Define skeleton connections - simplified MPII skeleton
        # Adjust based on your joint ordering in MPII dataset
        skeleton_pairs = [
            (0, 1),  # head to neck
            (1, 2),  # neck to right shoulder
            (1, 5),  # neck to left shoulder
            (2, 3),  # right shoulder to right elbow
            (3, 4),  # right elbow to right wrist
            (5, 6),  # left shoulder to left elbow
            (6, 7),  # left elbow to left wrist
            (1, 8),  # neck to hip
            (8, 9),  # hip to right hip
            (8, 12), # hip to left hip
            (9, 10), # right hip to right knee
            (10, 11), # right knee to right ankle
            (12, 13), # left hip to left knee
            (13, 14)  # left knee to left ankle
        ]
        
        # Draw skeleton lines connecting joints
        for pair in skeleton_pairs:
            if pair[0] < len(predicted_joints) and pair[1] < len(predicted_joints):
                plt.plot([predicted_joints[pair[0]][0], predicted_joints[pair[1]][0]],
                         [predicted_joints[pair[0]][1], predicted_joints[pair[1]][1]], 'b-', linewidth=2)
        
        plt.title('Pose Skeleton')
        plt.tight_layout()
        plt.show()

# Function to predict pose on new images
def predict_pose(image_path, joint_models, feature_scaler):
    # Load and preprocess image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image from {image_path}")
        return None
    
    # Resize for consistent HOG extraction
    img_resized = cv2.resize(img, (128, 256))
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    
    # Extract HOG features
    hog = cv2.HOGDescriptor((128, 256), (16, 16), (8, 8), (8, 8), 9)
    features = hog.compute(img_gray).flatten()
    
    # Scale features
    features_scaled = feature_scaler.transform(features.reshape(1, -1))
    
    # Predict joint positions
    predicted_joints = []
    for x_model, y_model in joint_models:
        x_pred = x_model.predict(features_scaled)[0]
        y_pred = y_model.predict(features_scaled)[0]
        predicted_joints.append((x_pred, y_pred))
    
    # Visualize
    plt.figure(figsize=(8, 16))
    plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
    
    # Draw predicted joints
    for joint in predicted_joints:
        plt.plot(joint[0], joint[1], 'ro', markersize=5)
    
    # Draw skeleton lines connecting joints
    skeleton_pairs = [
        (0, 1),  # head to neck
        (1, 2),  # neck to right shoulder
        (1, 5),  # neck to left shoulder
        (2, 3),  # right shoulder to right elbow
        (3, 4),  # right elbow to right wrist
        (5, 6),  # left shoulder to left elbow
        (6, 7),  # left elbow to left wrist
        (1, 8),  # neck to hip
        (8, 9),  # hip to right hip
        (8, 12), # hip to left hip
        (9, 10), # right hip to right knee
        (10, 11), # right knee to right ankle
        (12, 13), # left hip to left knee
        (13, 14)  # left knee to left ankle
    ]
    
    for pair in skeleton_pairs:
        if pair[0] < len(predicted_joints) and pair[1] < len(predicted_joints):
            plt.plot([predicted_joints[pair[0]][0], predicted_joints[pair[1]][0]],
                     [predicted_joints[pair[0]][1], predicted_joints[pair[1]][1]], 'b-', linewidth=2)
    
    plt.title('Pose Estimation')
    plt.tight_layout()
    plt.show()
    
    return predicted_joints

# Function to save models
def save_models(joint_models, feature_scaler, filepath='hog_svm_pose_model.pkl'):
    with open(filepath, 'wb') as f:
        pickle.dump({
            'joint_models': joint_models,
            'feature_scaler': feature_scaler
        }, f)
    print(f"Models saved to {filepath}")

# Function to load models
def load_models(filepath='hog_svm_pose_model.pkl'):
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data['joint_models'], data['feature_scaler']

# Main execution
if __name__ == "__main__":
    import time
    import pickle
    
    # 1. Load MPII dataset
    print("Loading MPII dataset...")
    mpii_data = load_mpii_dataset('mpii_human_pose_v1_u12_1.mat', './') 
    
    # Check if we got any data
    if len(mpii_data) == 0:
        print("No data loaded from MPII dataset. Check paths and file format.")
    else:
        # 2. Preprocess data
        print("\nPreprocessing images...")
        processed_mpii = preprocess_for_hog_svm(mpii_data)
        
        # 3. Extract HOG features
        print("\nExtracting HOG features...")
        X, y = extract_hog_features(processed_mpii)
        print(f"Feature matrix shape: {X.shape}")
        print(f"Labels matrix shape: {y.shape}")
        
        # 4. Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # 5. Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # 6. Train models
        print("\nTraining SVM models...")
        joint_models, errors = train_joint_models(X_train_scaled, y_train, X_test_scaled, y_test)
        
        # 7. Save models
        save_models(joint_models, scaler)
        
        # 8. Evaluate
        print("\nEvaluating models...")
        evaluate_and_visualize(processed_mpii, joint_models, scaler, num_samples=3)
        
        # 9. Test on specific image
        print("\nTesting on a new image...")
        test_img_path = processed_mpii[0]['original_path']  # Using first image as example
        predict_pose(test_img_path, joint_models, scaler)

Loading MPII dataset...
Loading annotations from mpii_human_pose_v1_u12_1.mat


IndexError: index 1 is out of bounds for axis 0 with size 1